In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('dataset/train_split_partially_preprocessed.csv')

In [ ]:
test_df = pd.read_csv('dataset/test_split_partially_preprocessed.csv')

In [ ]:
assert(len(train_df.columns) == len(test_df.columns))

In [ ]:
full_df = pd.concat([train_df, test_df])

In [ ]:
# Add speaks no english column (makes data analysis easier)
full_df['speaks_no_english'] = ~(full_df.speaksEnglish | full_df.speaksEnglishWell | full_df.speaksOnlyEnglish)
train_df['speaks_no_english'] = ~(train_df.speaksEnglish | train_df.speaksEnglishWell | train_df.speaksOnlyEnglish)
test_df['speaks_no_english'] = ~(test_df.speaksEnglish | test_df.speaksEnglishWell | test_df.speaksOnlyEnglish)

In [ ]:
full_df.dtypes.value_counts()

In [ ]:
full_df.INCWAGE_CPIU_2010.mean()

In [ ]:
def printColsValues(df, dfName, cols):
    print(f"Values for {dfName} ({len(df)} entries)\n")
    for col in cols:
        print(f'Information for column {col}:')
        if(df[col].dtype == 'float64' or df[col].dtype == 'int64'):
            print(df[col].describe())
        elif(df[col].dtype == 'object'):
            print(f'True: {df[col].values.sum()}, False: {(~df[col]).values.sum()}, Null: {df[col].isnull().values.sum()}')
        elif(df[col].dtype == 'bool'):
            print(f'True: {df[col].values.sum()}, False: {(~df[col]).values.sum()}')
        else:
            raise Exception(f'Unknown type {df[col].dtype}')
        print()

In [ ]:
# There was a bug in an old implementation of printColsValues. It caused errors for boolean columns with more true values than false values. This function finds such columns.
def findBugs(df, dfName, cols):
    for col in cols:
        if(df[col].dtype == 'object' or df[col].dtype == 'bool'):
            if(df[col].values.sum() >= (~df[col]).values.sum()):
                print("Error likely occured in", col)

In [ ]:
cols = [
    'AGE', 'INCWAGE_CPIU_2010', 'isFemale', 
    'isAmericanIndian', 'isAsian', 'isBlack', 'isPacificIslander', 'isWhite', 'isOtherRace',
    'hasHealthInsurance', 'hasPrivateHealthInsurance', 'hasPublicHealthInsurance',
    'isInSchool', 'isHispanic',
    'bornInUS', 'speaksEnglish', 'speaksOnlyEnglish', 'speaksEnglishWell',
    'isMarried', 'wasMarried', 'neverMarried',
    'sameSexMarriage', 'mixedRaceMarriage',
    'isSelfEmployed', 'isPrivateSector', 'isPublicSector', 'isUnpaidFamilyWorker',
    'noSchooling', 'maxGrade4',
       'maxGrade8', 'maxSomeHS', 'highSchoolDiploma', 'someCollege',
       'associatesDegree', 'bachelorsDegree', 'mastersDegree',
       'bachelorsPlusProfessionalDegree', 'doctoralDegree', 'has2ndDegree',
    
]

In [ ]:
for col in cols:
    if(full_df[col].isna().values.any()):
        print(f'Column {col} has NaN values')

In [ ]:
findBugs(full_df, '', cols)

In [ ]:
printColsValues(full_df, 'Full DF', cols)

In [ ]:
findBugs(train_df, '', cols)

In [ ]:
printColsValues(train_df, 'Train split', cols)

In [ ]:
findBugs(test_df, '', cols)

In [ ]:
printColsValues(test_df, 'Test split', cols)

In [ ]:
df_0to25 = full_df[full_df['INCWAGE_CPIU_2010'] <= 25000]
printColsValues(df_0to25, 'Under 25k salary', cols)

In [ ]:
findBugs(df_0to25, '', cols)

In [ ]:
df_25to50 = full_df[(full_df['INCWAGE_CPIU_2010'] > 25000) & (full_df['INCWAGE_CPIU_2010'] <= 50000)]
printColsValues(df_25to50, '25k to 50k salary', cols)

In [ ]:
findBugs(df_25to50, '', cols)

In [ ]:
df_50to75 = full_df[(full_df['INCWAGE_CPIU_2010'] > 50000) & (full_df['INCWAGE_CPIU_2010'] <= 75000)]
printColsValues(df_50to75, '50k to 75k salary', cols)

In [ ]:
findBugs(df_50to75, '', cols)

In [ ]:
df_over75 = full_df[(full_df['INCWAGE_CPIU_2010'] > 75000)]
printColsValues(df_over75, 'Over 75k salary', cols)

In [ ]:
findBugs(df_over75, '', cols)

In [ ]:
for col in cols:
    if full_df[col].dtype != 'bool':
        continue

    assert(not full_df[col].isna().values.any())

    print(f'Of the people for whom {col} is true, salary information is:')
    print(full_df[full_df[col]].INCWAGE_CPIU_2010.describe())
    print(f'Of the people for whom {col} is false, salary information is:')
    print(full_df[~full_df[col]].INCWAGE_CPIU_2010.describe())
    print("\n")

In [ ]:
assert(len(df_0to25) + len(df_25to50) + len(df_50to75) + len(df_over75) == len(full_df))

for col in cols:
    if full_df[col].dtype != 'bool':
        continue

    assert(not full_df[col].isna().values.any())

    lower = df_0to25[col].values.sum()
    lowerMid = df_25to50[col].values.sum()
    upperMid = df_50to75[col].values.sum()
    upper = df_over75[col].values.sum()

    totalNum = full_df[col].values.sum()
    assert(totalNum == lower + lowerMid + upperMid + upper)

    print(f'Of the people for whom {col} is true ({totalNum} people), salary information is:')
    print(f"Number making <=25k: {lower}, number making 25-50k: {lowerMid}, number making 50-75k: {upperMid}, number making >75k: {upper}")
    print(f'Of the people for whom {col} is false ({len(full_df) - totalNum}), salary information is:')
    print(f"Number making <=25k: {len(df_0to25) - lower}, number making 25-50k: {len(df_25to50) - lowerMid}, number making 50-75k: {len(df_50to75) - upperMid}, number making >75k: {len(df_over75) - upper}")
    print("\n")